# Agent Notebook 

# Import and Install ALL the DEPENDENCIES 
python version 3.8.10 is used 

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# MP HOLISTIC MODEL SETUP AND KEYPOINTS

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image,model):
    image= cv2.cvtColor(image,cv2.COLOR_BGR2RGB) # colour conersion is taken place 
    image.flags.writeable = False                  # image is not writeable
    results = model.process(image)                 
    image.flags.writeable = True                   # image is now writeable
    image= cv2.cvtColor(image,cv2.COLOR_RGB2BGR)  # 
    return image, results

In [ ]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [ ]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(90,120,20),thickness=1,circle_radius=1),
                                mp_drawing.DrawingSpec(color=(90,266,131),thickness=1,circle_radius=1))
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(90,42,20),thickness=3,circle_radius=4),
                                mp_drawing.DrawingSpec(color=(90,22,131),thickness=3,circle_radius=2))
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(90,42,20),thickness=3,circle_radius=4),
                                mp_drawing.DrawingSpec(color=(90,22,131),thickness=3,circle_radius=2))
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(90,112,20),thickness=3,circle_radius=4),
                                mp_drawing.DrawingSpec(color=(90,69,131),thickness=3,circle_radius=2))

In [ ]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #read the feed from the camera
        ret, frame = cap.read()
        image, results=mediapipe_detection(frame, model=holistic)
        draw_styled_landmarks(image ,results)
        #show the frame to the user 
        cv2.imshow('MAX', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# EXTRACT KEY PONTS

In [ ]:
def extract_keypoints(results): #for the extraction of keypoints from the results
    pose=np.array([res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmarks).flatten() if results.pose_landmarks else np.zeros(33*4)
    face=np.array([res.x, res.y, res.z] for res in results.face_landmarks.landmarks).flatten() if results.face_landmarks else np.zeros(468*3)
    lh=np.array([res.x, res.y, res.z] for res in results.left_hand_landmarks.landmarks).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh=np.array([res.x, res.y, res.z] for res in results.right_hand_landmarks.landmarks).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh]) #conatinate all the arrays

# SETUP 

In [ ]:
DATA_PATH =os.path.join ('DATASET') #dataset path
actions = np.array (['hello', 'thanks', 'iloveyou'])
no_sequences =30
sequence_length = 30


In [ ]:
for actions in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join (DATA_PATH, actions, str(sequence)))
        except:
            pass

# keypoints Values For Training and Testing

In [ ]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame in range(sequence_length):

                #read the feed from the camera
                ret, frame = cap.read()
                image, results=mediapipe_detection(frame, model=holistic)
                draw_styled_landmarks(image ,results)
                #show the frame to the user 
                cv2.imshow('MAX', image)
                if frame==0:
                    cv2.putText(image,'STARTING COLLECTION',(120,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.imshow('MAX', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'COLLECTING FRAME '+ str(frame),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),1,cv2.LINE_AA)
                key=extract_keypoints(results)
                npy_path=os.path.join (DATA_PATH, action, str(sequence), str(frame))
                np.save(npy_path, key)
                #np arrays are saved in the folders
        #to break or to stop the recording
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

# Preprocess Data and create labels and features 

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
sequence,label=[],[]
for action in actions:
    for sequnce in range(no_sequences):
        window=[]
        for fram in range(sequence_length):
            res= np.load(os.path.join (DATA_PATH, action, str(sequnce), str(fram)+'.npy'))
            window.append(res)
        sequence.append(window)
        label.append(label_map[action])

In [ ]:
x =np.array(sequence)
y= to_categorical(label).astype (int)
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size=0.05)

# Bob the build  and Train section

In [ ]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
lod_dir= os.path.join ('Logs')
tb_callback = TensorBoard(log_dir=lod_dir) 

In [ ]:
# building nueral network 
model= Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'], callbacks=[tb_callback])

In [ ]:
model.fit(x_train, y_train, epochs=2000, validation_data=(x_test, y_test), callbacks=[tb_callback])

# Predections

In [ ]:
res=model.predict(x_test)
actions[np.argmax(res[0])]
actions[np.argmax(y_test[0])]

# save weights

In [ ]:
model.save('action.h5')

In [ ]:
del model

In [ ]:
model.load.weights('action.h5')

# evaluation using Confusin MAtrix and accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(x_train)
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)
accuracy_score(ytrue, yhat)

# TESTING in REAL TIME    
 lets gooooo


In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob(res, actions, image, colors):
    output=image.copy()
    for num, prob in enumerate (res):
        cv2.rectangle (output, (0,60+num*40), (int (prob*100), 90+num*40), colors[num], -1)
        cv2.putText (output, actions[num], (0,85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    return output

In [ ]:
plt.figure(figsize=(18,10))
plt.imshow(prob(res[0], actions, image, colors))

In [ ]:
sequence=[]
sentence=[]
thresh=0.4 # percentage threshold



cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #read the feed from the camera
        ret, frame = cap.read()
        image, results=mediapipe_detection(frame, model=holistic)
        draw_styled_landmarks(image ,results)
        #show the frame to the user 
        key=extract_keypoints(results)
        sequence.append(key)
        sequence= sequence[-30:]
        if len (sequence)==30:
            res=model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            if res[np.argmax(res)]>thresh:
                if len (sentence)>0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
            if len (sentence)>5:
                sentence= sentence[-5:]
            
            image=prob(res, actions, image, colors) #for probability visualization
            
            #visualization logic
            cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2, cv2.LINE_AA)
        cv2.imshow('MAX', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()